In [1]:
import sys
sys.path.append("../")

In [2]:
import torch
import numpy as np
from samplers.DiffusiveGibbs import langevin_diffusive_gibbs_sampler_step
from utils import get_ground_truth_bnn, evaluate_test_nll, get_param_dim
from tqdm import tqdm

device = torch.device('cpu')
torch.set_default_tensor_type('torch.FloatTensor')

/Users/wenlinchen/miniconda3/envs/rdgs/lib/python3.9/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [3]:
d_x=20
d_h=25
s_n=0.1
dim = get_param_dim(d_x, d_h)
x_train, x_test, y_train, y_test, param_true, energy = get_ground_truth_bnn(d_x, d_h, s_n)

print("true_nll:", evaluate_test_nll(param_true, x_test, y_test, d_x, d_h, s_n))

true_nll: -0.8763405084609985


In [4]:
# smapler seed
seed = 1

np.random.seed(seed)
torch.manual_seed(seed)


In [5]:
num_samples = 150
num_chains = 2  # to save time, we run 2 independent DiGS samplers in parallel, each generating 75 samples
step_size = 1e-4
num_gibbs_sweeps_per_sample = 100
num_langevin_steps_per_sweep = 10  # 100x10=1000 Langevin steps per sample per time step

alpha_T = 0.1
alpha_1 = 0.9
T = 5

param = torch.randn([num_chains, dim]).to(device)

alpha_list = np.linspace(alpha_T, alpha_1, T)
std_list = np.sqrt(1.0 - alpha_list**2)

samples = []
with tqdm(range(num_samples//num_chains)) as T:
    for _ in T:
        for (alpha, std) in zip(alpha_list, std_list):
            # print(alpha, std)
            param = langevin_diffusive_gibbs_sampler_step(param, energy, alpha=alpha, std=std, num_gibbs_sweeps=num_gibbs_sweeps_per_sample, num_langevin_steps_per_sweep=num_langevin_steps_per_sweep, step_size=step_size, use_mala=True, device=device)
        samples.append(param.detach())
        nll = evaluate_test_nll(torch.cat(samples, dim=0), x_test, y_test, d_x, d_h, s_n)
        T.set_postfix({"nll": nll})

samples = torch.cat(samples, dim=0)
nll = evaluate_test_nll(samples, x_test, y_test, d_x, d_h, s_n)
    

100%|██████████| 75/75 [03:05<00:00,  2.48s/it, nll=0.189]


In [6]:
print("NLL:", nll)

NLL: 0.18860545754432678
